In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
world.reset()

In [ ]:
import numpy as np

In [ ]:
import getpass
user = getpass.getuser()

In [ ]:
from omni.isaac.kit import SimulationApp

usd_path = "/home/yizhou/Research/OpenAnyDrawer/scene0.usd" # f"omniverse://localhost/Users/{user}/test_franka2.usd"


simulation_app = SimulationApp({"headless": True, "open_usd": usd_path, "livesync_usd": usd_path})


In [ ]:
from omni.isaac.core import World
world = World()

In [ ]:
from open_env import OpenEnv

env = OpenEnv("/World/Game*/Franka", "/World/Game*/Franka/panda_hand")

In [ ]:
world.reset()

In [ ]:
env.start()

In [ ]:
world.scene.add(env.robots)

In [ ]:
# get grasp location
goal_pos, goal_rot = env.calculate_grasp_location(keyword = "handle_6", verticle = True)

In [ ]:
env.robots.get_dof_limits()[0]

# first stage: move the correct y,z, keep x far 

In [ ]:
goal_pos[...,0] -= 0.15

In [ ]:
goal_pos, goal_rot

In [ ]:
# bboxes_list = env.get_mesh_bboxes("handle_")
# center_list = [(e[1] + e[0]) / 2 for e in bboxes_list]

# center_list[0][0] -= 0.2
# center_list[1][0] -= 0.2

# goal_pos = np.array([list(e) for e in center_list], dtype=np.float32)
# goal_rot = np.array([[0, 0.70711, 0, 0.70711]], dtype=np.float32).repeat(2, axis = 0) # XYZW

# # goal_pos, goal_rot = hand_pos, hand_rot

In [ ]:
from usd_utils import calcuate_rotation_axis
from numpy_utils import quat_conjugate, quat_mul

In [ ]:
goal_rot

In [ ]:
env.move_hand_to_fast(goal_pos, goal_rot, world, max_step=300)

In [ ]:
# # stage
# for _ in range(300):
#     world.step(render=True)
    
#     # get end effector transforms
#     hand_pos, hand_rot = env.xforms.get_world_poses()
#     hand_rot = hand_rot[:,[1,2,3,0]] # WXYZ -> XYZW
    
#     if abs(quat_mul(goal_rot[0], quat_conjugate(hand_rot[0]))[3] - 1) < 0.03 and \
#         np.sqrt(np.sum((goal_pos[0] - hand_pos[0])**2)) < 0.01:
#         print(hand_rot)
#         print("Done rotation, position")
#         break

#     u = env.move_to_target(goal_pos, goal_rot)
#     u[:,[-2, -1]] = 0.05
#     env.robots.set_joint_position_targets(u)

# Second Stage: move to the grasp position

In [ ]:
goal_pos[...,0] += 0.15

# center_list[0][0] -= 0.1
# center_list[1][0] -= 0.1
# goal_pos = np.array([list(e) for e in center_list], dtype=np.float32)
# goal_rot = np.array([[0, 0.70711, 0, 0.70711]], dtype=np.float32).repeat(2, axis = 0) # XYZW

In [ ]:
goal_pos

In [ ]:
env.move_hand_to_slow(goal_pos, goal_rot, world, step = 60)

In [ ]:
# for i in range(60):
#     world.step(render=True)
    
#     u = env.move_to_target(goal_pos, goal_rot)
#     u[:,[-2, -1]] = 0.5 
#     env.robots.set_joint_position_targets(u)

# Third Stage: close gripper

In [ ]:
# env.move_hand_to_slow(goal_pos, goal_rot, world, open_gripper=False, step = 60)

In [ ]:
for i in range(60):
    world.step(render=True)
    
    u = env.move_to_target(goal_pos, goal_rot)
    u[:,[-2, -1]] = 0.5 - (0.5 - -1) / 60 * i
    env.robots.set_joint_position_targets(u)

# Forth Stage: Pull Open

1. Open from the right

In [ ]:
# rotate and pull
for i in range(60):
    world.step(render=True)
    
    target_pos, target_rot = env.calculate_pull_location(goal_pos, goal_rot, np.pi / 180 * i, 0.3)
    u = env.move_to_target(target_pos, target_rot)
    u[:,[-2, -1]] = -1
    env.robots.set_joint_position_targets(u)

2. Open from the left

In [ ]:
# rotate and pull
for i in range(10, 60):
    world.step(render=True)
    
    target_pos, target_rot = env.calculate_pull_location(goal_pos, goal_rot, np.pi / 180 * i, 0.3, clock_wise=True)
    u = env.move_to_target(target_pos, target_rot)
    u[:,[-2, -1]] = -1
    env.robots.set_joint_position_targets(u)

3. Pull the drawer out

In [ ]:
# pull only
for i in range(60):
    world.step(render=True)
    
    goal_pos[...,0] -= 2e-4 * i
    u = env.move_to_target(goal_pos, goal_rot)
    u[:,[-2, -1]] = -1
    env.robots.set_joint_position_targets(u)

In [ ]:
## cannot slerp
# end_pos, end_rot = env.calculate_pull_location(goal_pos, goal_rot, np.pi / 3, 0.3)
# env.move_hand_to_slow(end_pos, end_rot, world, open_gripper=False, step = 60)

In [ ]:
world.reset()
world.render()

In [ ]:
pause